In [8]:
# 📌 STEP 1: Import Libraries
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# 📌 STEP 2: Load Dataset
df = pd.read_csv('Sleep_health_and_lifestyle_dataset.csv')

# 📌 STEP 3: Preprocessing
df['Sleep Disorder'] = df['Sleep Disorder'].fillna('None')
df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})

# Convert Blood Pressure string (e.g., "120/80") to two numeric columns
df[['Systolic BP', 'Diastolic BP']] = df['Blood Pressure'].str.split('/', expand=True).astype(float)

# Heart Rate Category
df['HeartRateCategory'] = df['Heart Rate'].apply(lambda x: 0 if x < 60 else 1 if x <= 100 else 2)

# Encode Sleep Disorder
le = LabelEncoder()
df['Sleep_Disorder_Label'] = le.fit_transform(df['Sleep Disorder'])  # 0=Insomnia, 1=None, 2=Sleep Apnea

# Clean BMI column
df['BMI Category'] = df['BMI Category'].replace({
    'Normal': 1,
    'Normal Weight': 1,
    'Overweight': 2,
    'Obese': 3
}).infer_objects(copy=False)

# 📌 STEP 4: Define Feature Columns
features = [
    'Age', 'Gender', 'Sleep Duration', 'Quality of Sleep',
    'Physical Activity Level', 'Stress Level', 'Heart Rate',
    'Daily Steps'
]
X = df[features]

# 📌 STEP 5: Train-Test Split (1 time only)
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# 📌 STEP 6: Model Training & Saving

## 1. Sleep Disorder (Classification)
y1 = df['Sleep_Disorder_Label']
y1_train = y1[X_train.index]
model1 = RandomForestClassifier()
model1.fit(X_train, y1_train)
joblib.dump(model1, 'model_sleep.pkl')

## 2. Heart Rate Category (Classification)
y2 = df['HeartRateCategory']
y2_train = y2[X_train.index]
model2 = RandomForestClassifier()
model2.fit(X_train, y2_train)
joblib.dump(model2, 'model_heart.pkl')

## 3. BMI Category (Classification)
y3 = df['BMI Category']
y3_train = y3[X_train.index]
model3 = RandomForestClassifier()
model3.fit(X_train, y3_train)
joblib.dump(model3, 'model_bmi.pkl')

## 4. Systolic BP (Regression)
y4 = df['Systolic BP']
y4_train = y4[X_train.index]
model4 = RandomForestRegressor()
model4.fit(X_train, y4_train)
joblib.dump(model4, 'model_systolic_bp.pkl')

## 5. Diastolic BP (Regression)
y5 = df['Diastolic BP']
y5_train = y5[X_train.index]
model5 = RandomForestRegressor()
model5.fit(X_train, y5_train)
joblib.dump(model5, 'model_diastolic_bp.pkl')

## 6. Stress Level (Regression)
y6 = df['Stress Level']
y6_train = y6[X_train.index]
model6 = RandomForestRegressor()
model6.fit(X_train, y6_train)
joblib.dump(model6, 'model_stress.pkl')

## 7. Quality of Sleep (Regression)
y7 = df['Quality of Sleep']
y7_train = y7[X_train.index]
model7 = RandomForestRegressor()
model7.fit(X_train, y7_train)
joblib.dump(model7, 'model_qos.pkl')

print("✅ All models trained and saved successfully!")


C:\Users\anshi\AppData\Local\Temp\ipykernel_15832\489225029.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['BMI Category'] = df['BMI Category'].replace({


✅ All models trained and saved successfully!


In [12]:
from flask import Flask, request, jsonify
import joblib
import threading

# ─── Load All Models ────────────────────────────────────────────────────────────
model_sleep        = joblib.load('model_sleep.pkl')
model_heart        = joblib.load('model_heart.pkl')
model_bmi          = joblib.load('model_bmi.pkl')
model_systolic_bp  = joblib.load('model_systolic_bp.pkl')
model_diastolic_bp = joblib.load('model_diastolic_bp.pkl')
model_stress       = joblib.load('model_stress.pkl')
model_qos          = joblib.load('model_qos.pkl')

# ─── Flask Setup ───────────────────────────────────────────────────────────────
app = Flask(__name__)

def extract_features(data):
    return [
        data['Age'],
        1 if data['Gender'] == 'Male' else 0,
        data['Sleep Duration'],
        data['Quality of Sleep'],
        data['Physical Activity Level'],
        data['Stress Level'],
        data['Heart Rate'],
        data['Daily Steps']
    ]

@app.route('/predict-all', methods=['POST'])
def predict_all():
    data = request.get_json()
    feat = [extract_features(data)]
    
    # 1. Sleep Disorder
    sd = model_sleep.predict(feat)[0]
    sd_map = {0: "Insomnia", 1: "None", 2: "Sleep Apnea"}
    
    # 2. Heart Rate Category
    hr = model_heart.predict(feat)[0]
    hr_map = {0: "Low", 1: "Normal", 2: "High"}
    
    # 3. BMI Category
    bmi = model_bmi.predict(feat)[0]
    bmi_map = {1: "Normal", 2: "Overweight", 3: "Obese"}
    
    # 4. Systolic BP
    sbp = model_systolic_bp.predict(feat)[0]
    # 5. Diastolic BP
    dbp = model_diastolic_bp.predict(feat)[0]
    
    # 6. Stress Level (regression)
    st = model_stress.predict(feat)[0]
    # 7. Quality of Sleep (regression)
    qos = model_qos.predict(feat)[0]
    
    return jsonify({
        'sleep_disorder':        sd_map.get(sd, "Unknown"),
        'heart_rate_category':   hr_map.get(hr, "Unknown"),
        'bmi_category':          bmi_map.get(bmi, "Unknown"),
        'systolic_bp':           round(float(sbp), 1),
        'diastolic_bp':          round(float(dbp), 1),
        'predicted_stress':      round(float(st), 1),
        'predicted_quality_of_sleep': round(float(qos), 1)
    })

# ─── Run Flask in Background ─────────────────────────────────────────────────
def run_app():
    app.run(port=5000, debug=True, use_reloader=False)

threading.Thread(target=run_app).start()


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
